In [1]:
"""
Function to compute the execution time.

"""
from datetime import timedelta
import time
def get_time(f, kwargs={}):
    times = []
    result = None
    for i in range(1):
        print("\nmeasure",i+1)
        t = time.time()
        result = f(**kwargs)
        times.append(time.time()-t)
        
    print("\nall times:", times)
    a = min(times)
    b = max(times)
    print("times' outliers:",a,"&",b)
    t = abs(sum(times)-a-b)
    print("benchmark time",str(t)+"s","or", timedelta(seconds=t))
    return result

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
from pymeos import *
import numpy as np
from pymeos.temporal import interpolation
pymeos_initialize()

In [3]:



def load_ships_trajectories(df_):
    df = df_.copy()
    df = df.dropna()
    df["Timestamp"] = pd.to_datetime(df["Timestamp"],format="mixed")
    df = df.sort_values(by="Timestamp")
    df = df.drop_duplicates(["Timestamp"])
    print('point number',len(df))
    df['point'] = df.apply(lambda row: TGeogPointInst(point=(row['Latitude'], row['Longitude']), timestamp=row['Timestamp']),axis=1)
    df['SOG'] = df.apply(lambda row: TFloatInst(value=row['SOG'] or 0 , timestamp=row['Timestamp']), axis=1)
    df['ROT'] = df.apply(lambda row: TFloatInst(value=row['ROT']or 0, timestamp=row['Timestamp']), axis=1)
    df['Heading'] = df.apply(lambda row: TFloatInst(value=row['Heading'] or 0, timestamp=row['Timestamp']), axis=1)
    df.drop(['Latitude', 'Longitude'], axis=1, inplace=True)
    trajectories = df.groupby('id').aggregate(
        {
            'point': lambda x: TGeogPointSeq.from_instants(x, lower_inc=True,upper_inc=True,interpolation=interpolation.TInterpolation.DISCRETE),
            # 'SOG':  lambda x: TFloatSeq.from_instants(x, lower_inc=True,upper_inc=True),
            # 'ROT': lambda x: TFloatSeq.from_instants(x, lower_inc=True,upper_inc=True),
            # 'Heading': lambda x: TFloatSeq.from_instants(x, lower_inc=True,upper_inc=True)
        }
    ).rename({'point': 'trajectory'}, axis=1)
    return trajectories

def load_cars_trajectories(df_):
    df = df_.copy()
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], format="mixed")
    print("done 1!")
    df = df.sort_values(by="Timestamp")
    df = df.drop_duplicates(["Timestamp"])
    df['point'] = df.apply(lambda row: TGeogPointInst(point=(row['Latitude'], row['Longitude']), timestamp=row['Timestamp']),axis=1)
    df.drop(['Latitude', 'Longitude'], axis=1, inplace=True)
    trajectories = df.groupby('id').aggregate(
    {
        'point': TGeogPointSeq.from_instants,
    }
    ).rename({'point': 'trajectory'}, axis=1)
    return trajectories

# Ships Trajectories

In [4]:
SIZE = "sample"
PATH='../../datasets/ships_'+SIZE+'.csv'

df = pd.read_csv(PATH)
trajectories = get_time(load_ships_trajectories,{'df_':df})


measure 1
point number 18139

all times: [4.800894021987915]
times' outliers: 4.800894021987915 & 4.800894021987915
benchmark time 4.800894021987915s or 0:00:04.800894


In [48]:
print(TGeogPointInst(point=(40,68), timestamp=pd.to_datetime("25-03-2000")))

0101000020E610000000000000000044400000000000005140@2000-03-25 00:00:00+01


/tmp/ipykernel_6413/2169413057.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  print(TGeogPointInst(point=(40,68), timestamp=pd.to_datetime("25-03-2000")))


In [5]:
trajectories

,trajectory
id,
218768000,{0101000020E610000091F2936A9F2E4B40545227A0890...
219015579,{0101000020E6100000C02154A9D9FB4B40A305685BCD0...


In [6]:
size = len(trajectories)
tt_size = 0
for i in range(size):
    tt_size+=trajectories["trajectory"].values[i].num_instants()
tt_size

18139

In [7]:
tolerance = 1/111139
compressed = trajectories.copy()
compressed["trajectory"] = compressed["trajectory"].apply(lambda x:x.simplify(tolerance))
tt_size = 0
for i in range(size):
    tt_size+=compressed["trajectory"].values[i].num_instants()
tt_size

18139

In [54]:
compressed

,trajectory
id,
218768000,[0101000020E610000091F2936A9F2E4B40545227A0890...
219015579,[0101000020E6100000C02154A9D9FB4B40A305685BCD0...


# cars trajectories

In [10]:
PATH='../datasets/cars_'+SIZE+'.csv'
df = pd.read_csv(PATH)
print("done 0!")

trajectories = load_cars_trajectories(df)

done 0!
done 1!


In [ ]:
trajectories